# Отбор признаков

## 1. Подготовка данных

### 1.1 Выполним импорт датаеета и библиотек

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


data = pd.read_excel('data/data_ford_price.xlsx')

# Оставляем только переменные типа int и float
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]

# Избавляемся от пропусков
data.dropna(inplace = True)


### 1.2. Выполним дополнительную очистку строк по целевому признаку *price* для более корретной работы

In [ ]:
# Удаяем строки, где Price < 100
data = data.drop(data[data['price'] < 100].index)

# Удаяем строки, где Price > 70000
data = data.drop(data[data['price'] > 70000].index)

### 1.3. Разделим данные - отделим целевой признак *price*

In [ ]:
# Целевой признак
y = data['price']

# Нецелевые признаки
x = data.drop(columns='price')

## 2. Выполним отбор признаков 2-мя методами:
- RFE
- SelectKBest

### 2.1. Обор признаков методом RFE

In [ ]:
from sklearn.feature_selection import RFE
 
# Модель линейной регрессии
estimator = LinearRegression()

# Используем метод RFE (количество признаков - 3)
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(x, y)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

### 2.2. Обор признаков методом SelectKBest

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# Используем метод RFE (количество признаков - 3, f_regression - для числовых признаков на входе и выходе)
selector = SelectKBest(f_regression, k=3)
selector.fit(x, y)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

### 2.3. Разделяем датасеты на тренировочные и обучаемые

In [ ]:
# Набор признаков по методу RFE
X_train_rfe, X_test_rfe, y_train_rfe, y_test_rfe = train_test_split(x[['year', 'cylinders', 'lat']], y, test_size=0.3, random_state=42)

In [ ]:
# Набор признаков по методу SelectKBest
X_train_skb, X_test_skb, y_train_skb, y_test_skb = train_test_split(x[['year', 'cylinders', 'odometer']], y, test_size=0.3, random_state=42)

## 3. Обучение моделей

In [ ]:
# Модель по методу RFE
model_rfe = LinearRegression()

In [ ]:
# Модель по методу SelectKBest
model_skb = LinearRegression()

In [ ]:
# Обучение модели по методу RFE (признаки 'year', 'cylinders', 'lat')

model_rfe.fit(X_train_rfe, y_train_rfe)

LinearRegression()

In [ ]:
# Обучение модели по методу SelectKBest (признаки 'year', 'cylinders', 'odometer')

model_skb.fit(X_train_skb, y_train_skb)

LinearRegression()

## 4. Проверка метрик (MAE, R2, MAPE)

In [ ]:
# Предсказанныq целевой признак Price по методу RFE
y_predicted_rfe = model_rfe.predict(X_test_rfe)

# Метрики
mae = round(mean_absolute_error(y_test_rfe, y_predicted_rfe),3)
mape = round(mean_absolute_percentage_error(y_test_rfe, y_predicted_rfe),3)
R2 = round(r2_score(y_test_rfe, y_predicted_rfe),3)
print('MAE: %.3f' % mae, "\n"
      'MAPE: %.3f' % mape, "\n"
      'R-2 score: %.3f' % R2)

MAE: 4962.876 
MAPE: 0.869 
R-2 score: 0.590


In [ ]:
# Предсказанный целевой признак Price по методу SelectKBest
y_predicted_skb = model_skb.predict(X_test_skb)
 
# Метрики
mae = round(mean_absolute_error(y_test_skb, y_predicted_skb),3)
mape = round(mean_absolute_percentage_error(y_test_skb, y_predicted_skb),3)
R2 = round(r2_score(y_test_skb, y_predicted_skb),3)
print('MAE: %.3f' % mae, "\n"
      'MAPE: %.3f' % mape, "\n"
      'R-2 score: %.3f' % R2)

MAE: 4607.154 
MAPE: 0.809 
R-2 score: 0.643


## Выводы:

**1. Согласно заданию - проведен сравнительный анализ моделей линейной регрессии на основе отобранных признаков по двум методам:**

- **Метод рекурсивного исключения признаков (RFE)** - выбраны 3 признака ['year', 'cylinders', 'lat'],

- **Метод рекурсивного исключения признаков (RFE)** - выбраны 3 признака ['year', 'cylinders', 'odometer']

Можно заметить, что 2 из 3х признаков совпадают по двум методам

**2. Обучены 2 модели линейной регрессии:**

*Метрики на обеих моедля показывают не самый лучший результат - необходимо провести нормализацию/стандартизацию данных, проанализировать признаки, увеличить количество или создать новые.*

*Так как цель задания была проверить какой метод отбора признаков лучше, данные преобразования не были проведены*

### На основании полученных метрик моджем сделать вывод, что на признаках ['year', 'cylinders', 'odometer'] отобранных методом SelectKBest модель работает лучше
